In [1]:
# домашняя работа 

In [2]:
# 1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. 
#    Использовать датасет 
#    https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
# 2. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию 
#    для оценки качества моделей
# 3. Построить случайный лес, вывести важность признаков
# 4. Обучить стекинг как минимум 3х моделей, использовать 
#    хотя бы 1 линейную модель и 1 нелинейную
# 5. Для валидации модели 2-го уровня использовать 
#    отдельный hold-out датасет, как на занятии
# 6. Показать, что использование ансамблей моделей действительно 
#    улучшает качество (стекинг vs другие модели сравнивать на hold-out)
# 7. В качестве решения: Jupyter notebook с кодом, комментариями 
#    и графиками, ссылка на гитхаб


In [158]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split

In [271]:
import warnings
warnings.filterwarnings('ignore')

In [159]:
data = pd.read_csv('train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [160]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [161]:
# Заполнить средними значениеми
def put_mean_to_na_param(dataset, list_columns):
    for i in list_columns:
        dataset[i].fillna(dataset[i].mean(), inplace=True)

In [162]:
# Заполнить значением
def put_value_to_na_param(dataset, list_columns, value=0 ):
    for i in list_columns:
        dataset[i].fillna(value, inplace=True)

In [163]:
# Заполнить самым распространенным значением
def put_most_common_meaning_to_na_param(dataset, list_columns):
    for i in list_columns:
        dataset[i].fillna(dataset[i].value_counts()[0], inplace=True)

In [164]:
# Заполнить значения NaN значениеями из другой колонки
def put_param_from_col_src_to_na_param(dataset, col_dest, col_src): 
    dataset[col_dest]=dataset.apply(lambda x: x[col_src] if x[col_dest] is not None else x[col_dest], axis=1 )
            

In [165]:
# Закодировать nominal категориальную переменную с помощью "one hot encoding"  (используем реализацию из лекций)
def get_one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df 

# Закодировать ordered категориальную переменную используя числовой порядок
def get_ordered_col(dataset, col, dict_col_params):
    """
    В функцию передается датасет, имя колонки и словарь {'значение_параметра_в_колонке': числовое_значение_для_замены }
    """
    dataset[col + '_order'] = dataset.apply(lambda x: dict_col_params[x[col]],axis=1)
    return dataset

def get_ordered_cols(dataset, cols, dict_col_params):
    for col in cols:
        res = get_ordered_col(dataset, col, dict_col_params)
    return res.drop(columns=cols)     

dict_label_5_1 = {'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'Po': 0}
dict_label_5_2 = {'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'NA': 0}
dict_label_5_3 = {'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'NA': 0} 
dict_label_6_1 = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'NA': 0}
dict_label_7_1 = {'GLQ': 6, 'ALQ': 5, 'BLQ': 4, 'Rec': 3, 'LwQ': 2, 'Unf': 1, 'NA': 0} 


In [166]:
# функция подготовки данных
def get_data(dataset):
    put_value_to_na_param(dataset, ['MiscFeature','Alley','Fence','BsmtQual','PoolQC','BsmtCond',\
                                    'BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu',\
                                    'GarageType','GarageFinish','GarageQual','GarageCond'], 'NA' )
    
    put_value_to_na_param(dataset, ['MasVnrArea'], 0)
    put_value_to_na_param(dataset, ['MasVnrType'], 'none')                      
    put_mean_to_na_param(dataset, ['LotFrontage'])
    put_most_common_meaning_to_na_param(dataset, ['Electrical'])   
    put_param_from_col_src_to_na_param(dataset, 'GarageYrBlt', 'YearRemodAdd')     
    
    data_raw = dataset.copy()
    data_raw = get_one_hot(dataset, ['MiscFeature', 'Fence', 'Alley', 'Street', 'LotShape', 'LandContour', 'Utilities',\
                                     'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', 'Foundation', 'Heating', \
                                     'CentralAir', 'Electrical', 'PavedDrive', 'MSZoning','MSSubClass','Neighborhood','Condition1',\
                                     'Condition2','HouseStyle','RoofStyle','RoofMatl','Exterior1st',\
                                     'Exterior2nd','Functional','GarageType','GarageFinish',\
                                     'SaleType','SaleCondition'])
    
    data_raw = get_ordered_cols(data_raw, ['ExterQual', 'ExterCond', 'HeatingQC', 'KitchenQual'], dict_label_5_1)
    data_raw = get_ordered_cols(data_raw, ['PoolQC'], dict_label_5_2)
    data_raw = get_ordered_cols(data_raw, ['BsmtExposure'], dict_label_5_3)
    data_raw = get_ordered_cols(data_raw, ['BsmtQual', 'BsmtCond', 'FireplaceQu', 'GarageQual', 'GarageCond'] , dict_label_6_1)
    data_raw = get_ordered_cols(data_raw, ['BsmtFinType1', 'BsmtFinType2'] , dict_label_7_1) 
    
    y = data_raw['SalePrice'].copy()
    #data_raw= data_raw.drop(columns=['SaleCategory'])
    columns_to_drop = ['SalePrice', 'MSZoning','MSSubClass','Neighborhood','Condition1',\
                       'Condition2','HouseStyle','RoofStyle','RoofMatl','Exterior1st',\
                       'Exterior2nd','Functional','GarageType','GarageFinish',\
                       'SaleType','SaleCondition','MiscFeature', 'Fence', 'Alley',\
                       'Street', 'LotShape', 'LandContour', 'Utilities',\
                       'LotConfig', 'LandSlope', 'BldgType', 'MasVnrType', \
                       'Foundation', 'Heating', 'CentralAir', 'Electrical', 'PavedDrive']
    
    data_raw = data_raw.drop(columns=columns_to_drop)
    
        
    
    return data_raw, y
    

In [167]:
X_full, y_full = get_data(data)

In [168]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 263 entries, Id to BsmtFinType2_order
dtypes: float64(2), int64(47), uint8(214)
memory usage: 864.1 KB


In [169]:
X_full.head()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,KitchenQual_order,PoolQC_order,BsmtExposure_order,BsmtQual_order,BsmtCond_order,FireplaceQu_order,GarageQual_order,GarageCond_order,BsmtFinType1_order,BsmtFinType2_order
0,1,65.0,8450,7,5,2003,2003,196.0,706,0,...,3,0,1,4,3,0,3,3,6,1
1,2,80.0,9600,6,8,1976,1976,0.0,978,0,...,2,0,4,4,3,3,3,3,5,1
2,3,68.0,11250,7,5,2001,2002,162.0,486,0,...,3,0,2,4,3,3,3,3,6,1
3,4,60.0,9550,7,5,1915,1970,0.0,216,0,...,3,0,1,3,4,4,3,3,5,1
4,5,84.0,14260,8,5,2000,2000,350.0,655,0,...,3,0,3,4,3,3,3,3,6,1


In [240]:
# X и y общая выборка для моделей 1 уровня 
# X_out,y_out - выборка для проверки модели второго уровня

X, X_out, y, y_out = train_test_split(X_full.drop(columns=['Id']), y_full.values, test_size=0.1 ,random_state=42)

In [241]:
from sklearn.preprocessing import StandardScaler

In [242]:
sc = StandardScaler().fit(X)
X_keys = X.columns
X = sc.transform(X)
X_out = sc.transform(X_out)
X = pd.DataFrame(X, columns=X_keys)
X_out = pd.DataFrame(X_out, columns=X_keys)

In [243]:
# 3. Построить случайный лес, вывести важность признаков

from sklearn.ensemble import RandomForestRegressor

In [244]:
clf = RandomForestRegressor()
clf.fit(X,y)

feature_importances= {}
for col, val in zip(X.columns, clf.feature_importances_):
    feature_importances[col] = val
    
sorted_x = sorted(feature_importances.items(), key=lambda kv: kv[1], reverse=True)
sorted_x[:20]

[('OverallQual', 0.5751315657246734),
 ('GrLivArea', 0.10643824571941754),
 ('TotalBsmtSF', 0.03785147859618231),
 ('1stFlrSF', 0.03176891058112213),
 ('BsmtFinSF1', 0.027323343958554663),
 ('GarageCars', 0.02024748063238487),
 ('GarageArea', 0.019751293433822533),
 ('LotArea', 0.015185572420291834),
 ('2ndFlrSF', 0.013286837227311976),
 ('BsmtQual_order', 0.011630464289829067),
 ('KitchenQual_order', 0.007989839213890548),
 ('YearBuilt', 0.0076778124532431834),
 ('TotRmsAbvGrd', 0.006981053975069114),
 ('MasVnrArea', 0.006338698731625026),
 ('GarageYrBlt', 0.0062187610247281),
 ('LotFrontage', 0.005743704438036547),
 ('WoodDeckSF', 0.005312735138972856),
 ('BsmtUnfSF', 0.00520612672332285),
 ('OpenPorchSF', 0.004816361066619848),
 ('OverallCond', 0.004597161156202932)]

In [245]:
# 4. Обучить стекинг как минимум 3х моделей, использовать 
#    хотя бы 1 линейную модель и 1 нелинейную
# 5. Для валидации модели 2-го уровня использовать 
#    отдельный hold-out датасет, как на занятии

In [246]:
# Для стекинга используем следующие типы моделей
# модели первого уровня
# 1. SGDRegressor
# 2. Ridge
# 3. KNeighborsRegressor
# модель второго уровня
# 1. LinearRegression

In [261]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [262]:
def make_stacking(X, y, dict_model, meta_model, param_model_GS, param_meta_model):
    # получаем длину фолда
    len_fold = (len(X) // len(dict_model.keys()))+1
    kf = KFold(n_splits=len(dict_model.keys()), shuffle=True)
    result_fold = {}
    i = 0
    for train_index, test_index in kf.split(X):
        full_test_index_l = list(test_index)
        train_index_l =  list(train_index)
        # для того что бы данные для мета модели были полные
        # выравниваем длину фолдов
        # переносим избыток индексов из train в test
        if(len_fold > len(test_index)):
            # получим номера индексов в train  для переноса в test   
            add_num = random.sample(range(len(train_index)), len_fold > len(test_index))
            # получим индексы из train
            list_change_elem = train_index[add_num]
            # пройдем по списку индексов 
            for k in list_change_elem:
                # удалим индекс из train
                train_index_l.remove(k)
                #  добавим индекс в test
                full_test_index_l = full_test_index_l + list([k,])
        # -----------------------        
        # формируем выборки данных для фолдов
        k = list(dict_model.keys())[i]
        result_fold[k] = {'X_train': X.iloc[train_index_l], 'X_test': X.iloc[full_test_index_l],
                          'y_train': y[train_index_l], 'y_test': y[full_test_index_l]}
        # ------------------------
        i+=1
    result_model ={}
    # проходим по списку моделей 1 уровня 
    for j in dict_model.keys():
        # обучаем 
        dict_model[j] = GridSearchCV(dict_model[j], param_model_GS[j], cv=5).fit(result_fold[j]['X_train'], result_fold[j]['y_train']).best_estimator_
        # предсказания помещаем в словарь с данными для мета модели
        result_model[j] = dict_model[j].predict(result_fold[j]['X_test'])
    df_result = pd.DataFrame.from_dict(result_model)
    # обучаем мета модел
    meta_model =  GridSearchCV(meta_model, param_meta_model, cv=5).fit(df_result,result_fold[j]['y_test']).best_estimator_
    return dict_model, meta_model

In [266]:
def make_blanding(X, y, dict_model, meta_model,param_model_GS, param_meta_model):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)
    result_model ={}
    for i in dict_model.keys():
        dict_model[i] = GridSearchCV(dict_model[i], param_model_GS[i], cv=5).fit(X_train, y_train).best_estimator_
        result_model[i] = dict_model[i].predict(X_test)
    df_result = pd.DataFrame.from_dict(result_model)
    meta_model = GridSearchCV(meta_model, param_meta_model, cv=5).fit(df_result,y_test).best_estimator_
    return dict_model, meta_model

In [250]:
def predict_result(X, dict_model, meta_model):
    result_model ={}
    for i in dict_model.keys():
        result_model[i] = dict_model[i].predict(X)
    df_result = pd.DataFrame.from_dict(result_model)
    result = meta_model.predict(df_result) 
    return result

In [272]:
param_model_GS = {'Ridge':{'alpha':[0.1,0.5,1]}, 
              'Lasso':{'alpha':[0.1,0.5,1]}, 
              'KNeighborsRegressor':{'n_neighbors':[3,5,7,9]},
              'LinearSVR':{'C':[0.1,0.5,1], 'loss':['epsilon_insensitive', 'squared_epsilon_insensitive']},
              'DecisionTreeRegressor':{'max_depth':[3,5,7,9]},                                    
              'LinearRegression':{},
              'SGDRegressor':{'loss':['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
                              'penalty':['none', 'l2', 'l1','elasticnet'] 
                             }
             }



dict_model = {}
#dict_model['KNeighborsRegressor'] = KNeighborsRegressor()
dict_model['Ridge'] = Ridge()
dict_model['Lasso'] = Lasso()
#dict_model['LinearSVR'] = LinearSVR()
#dict_model['SGDRegressor'] = SGDRegressor()
dict_model['DecisionTreeRegressor'] = DecisionTreeRegressor()
meta_model = LinearRegression()


blending_dict_model, blending_meta_model = make_blanding(X, y, dict_model, meta_model,param_model_GS, param_model_GS['LinearRegression'])
blending_result = predict_result(X_out, blending_dict_model, blending_meta_model)

stacking_dict_model, stacking_meta_model = make_stacking(X, y, dict_model, meta_model,param_model_GS, param_model_GS['LinearRegression'])
stacking_result = predict_result(X_out, stacking_dict_model, stacking_meta_model)




In [274]:
blending_dict_model

{'DecisionTreeRegressor': DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
 'Lasso': Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False),
 'Ridge': Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001)}

In [275]:
# сравнение моделей

In [286]:
from sklearn.metrics import mean_absolute_error


In [281]:
Rg  = GridSearchCV(Ridge(), param_model_GS['Ridge'], cv=5).fit(X,y).best_estimator_.predict(X_out)
Ls  = GridSearchCV(Lasso(), param_model_GS['Lasso'], cv=5).fit(X,y).best_estimator_.predict(X_out)
KNR = GridSearchCV(KNeighborsRegressor(), param_model_GS['KNeighborsRegressor'], cv=5).fit(X,y).best_estimator_.predict(X_out)
SVR = GridSearchCV(LinearSVR(), param_model_GS['LinearSVR'], cv=5).fit(X,y).best_estimator_.predict(X_out)
DTR = GridSearchCV(DecisionTreeRegressor(), param_model_GS['DecisionTreeRegressor'], cv=5).fit(X,y).best_estimator_.predict(X_out)
SGD = GridSearchCV(SGDRegressor(), param_model_GS['SGDRegressor'], cv=5).fit(X,y).best_estimator_.predict(X_out)

In [287]:
print('blending:', mean_absolute_error(blending_result,y_out))
print('stacking:', mean_absolute_error(stacking_result,y_out))
print('ridge   :', mean_absolute_error(Rg,y_out))
print('lasso   :', mean_absolute_error(Ls,y_out))
print('KNR     :', mean_absolute_error(KNR,y_out))
print('SVR     :', mean_absolute_error(SVR,y_out))
print('DTR     :', mean_absolute_error(DTR,y_out))
print('SGD     :', mean_absolute_error(SGD,y_out))

blending: 18866.94580574168
stacking: 30565.642655267406
ridge   : 20333.968734261707
lasso   : 20514.189099687002
KNR     : 27300.959665144597
SVR     : 20276.749831743124
DTR     : 21178.35256398122
SGD     : 182859.5691885613


In [283]:
from sklearn.metrics import r2_score

In [288]:
print('blending:', r2_score(blending_result,y_out))
print('stacking:', r2_score(stacking_result,y_out))
print('ridge   :', r2_score(Rg,y_out))
print('lasso   :', r2_score(Ls,y_out))
print('KNR     :', r2_score(KNR,y_out))
print('SVR     :', r2_score(SVR,y_out))
print('DTR     :', r2_score(DTR,y_out))
print('SGD     :', r2_score(SGD,y_out))

blending: 0.8223412102003974
stacking: 0.6430009431226187
ridge   : 0.8605925614126352
lasso   : 0.8604944146358682
KNR     : 0.2301631689014455
SVR     : 0.8596769707682002
DTR     : 0.8540181978839124
SGD     : -128234789208.7641


In [259]:
# 

In [260]:
z = [1,2,3] + [4]

In [189]:
z.remove(1)

In [190]:
z

[2, 3, 4]